# confidence

> Estimating confidence that the suggested probabilities are correct.

In [ ]:
#| default_exp confidence

In [ ]:
#| hide
## Google Colab / Installs

In [ ]:
#| export
import warnings
warnings.filterwarnings('ignore')
import requests
from bs4 import BeautifulSoup
import enchant
import re
import random
from collections import Counter
from fastai.text.all import *
from pseudometer.data import *
import hashlib
import pickle

from sklearn.svm import OneClassSVM
from sklearn.feature_extraction.text import TfidfVectorizer

## One-Class Classifier

In [ ]:
#| hide
d_dl = process_data(path, train_sources, k, min_words, max_words, ignore_text, ignore_common, 
            ignore_filenames, max_links)

https://www.newscientist.com/ link   28/  33 | https://www.facebook.com/newscientist = Rejected                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [ ]:
train_sci = []
train_pse = []
test_sci = []
test_pse = []
for link in d_dl:
    text = d_dl[link][0]
    cat = d_dl[link][2]
    r = random.randint(0,5)
    if cat == 'science' and r != 0:
        train_sci.append(text)
    elif cat == 'science' and r == 0:
        test_sci.append(text)
    elif cat == 'pseudoscience' and r != 0:
        train_pse.append(text)
    elif cat == 'pseudoscience' and r == 0:
        test_pse.append(text)

In [ ]:
def normalize(arr):
    norm = []
    for val in arr:
        norm.append((val-np.min(arr))/(np.max(arr)-np.min(arr)))
    return norm

In [ ]:
train_sets = {'science':train_sci, 'pseudoscience':train_pse}
test_sets = {'science':test_sci, 'pseudoscience':test_pse}
norms = {}
vectorizer = TfidfVectorizer()
for cat_train in train_sets:
    for cat_test in test_sets:
        train_set = train_sets[cat_train]
        test_set = test_sets[cat_test]
        train_vectors = vectorizer.fit_transform(train_set)
        test_vectors = vectorizer.transform(test_set)

        model = OneClassSVM(gamma='auto')
        model.fit(train_vectors)

        test_predictions = model.predict(test_vectors)
        norm = normalize(test_predictions)
        norms[cat_train+', '+cat_test] = [np.round(np.mean(norm),2),np.round(np.std(norm),2)]

norms

{'science, science': [0.44, 0.5],
 'science, pseudoscience': [0.27, 0.44],
 'pseudoscience, science': [0.47, 0.5],
 'pseudoscience, pseudoscience': [0.63, 0.48]}

In [ ]:
#learn = load_learner('models/2022.11.28 Model.pth', cpu=False)